In [1]:
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz as pt
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException 
import os as oss
import coloredlogs

In [2]:
from docopt import docopt
from trading_botops import get_state
from trading_botagent import Agent
from trading_botmethods import evaluate_model
from trading_botutils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)


C:\Users\shree\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shree\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shree\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shree\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
tz = pt.timezone('Asia/Kolkata')

In [24]:
ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [26]:
driver = webdriver.Chrome(options = options)

In [44]:
def Real(url,count):
    if count == 0:
        driver.get(url)
    else:
        driver.refresh()
        time.sleep(15)
    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    return live 

In [ ]:
def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args.ticker + '.NS'
    price = []
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state

In [ ]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
    action = agent.act(state, is_eval=True)
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))
        elif action == 2 and len(agent.inventory) > 0:
            bought_price = agent.inventory.pop(0)
            delta = data[t] - bought_price
            reward = delta
            total_profit += delta
            history.append((data[t], "SELL"))
            if debug:
                logging.debug("Sell at: {} | Position: {}".format(format_currency(data[t]), format_position(data[t] - bought_price)))
        else:
            history.append((data[t], "HOLD"))
            agent.memory.append((state, action, reward, next_state))
            return total_profit
        if __name__ == "__main__":
            parser = argparse.ArgumentParser()
            parser.add_argument('ticker', help = 'ticker')
            args = parser.parse_args()
            coloredlogs.install(level="DEBUG")
            switch_k_backend_device()

    try:
        main(args)
    except KeyboardInterrupt:
        print("Aborted") 
                
        

            
   